In [8]:
#Step1:Convert original file into jpg in different folders.
import os
from skimage import io
import torchvision.datasets.mnist as mnist

train_set = (
mnist.read_image_file('gzip\emnist-mnist-train-images-idx3-ubyte\emnist-mnist-train-images-idx3-ubyte'),
mnist.read_label_file('gzip\emnist-mnist-train-labels-idx1-ubyte\emnist-mnist-train-labels-idx1-ubyte')
)
test_set = (
    mnist.read_image_file('gzip\emnist-mnist-test-images-idx3-ubyte\emnist-mnist-test-images-idx3-ubyte'),
    mnist.read_label_file('gzip\emnist-mnist-test-labels-idx1-ubyte\emnist-mnist-test-labels-idx1-ubyte')
)
#They are in 'tensor' type.

#print(train_set[0][1])
# 从原始数据 到 jpg图片 顺便装到文件夹里面
def convert_to_img(train=True):
    
    if train:  # 如果是训练数据
        # 注意这里路径的写法，对就是/符号
        f = open('train.txt', 'w')
        data_path ='train/' # 好像可以删去左边的/
        # 如果不存在就新建
        if not os.path.exists(data_path):
            os.makedirs(data_path)
            # enumerate将可遍历对象 组合成索引 可加参数start=2 索引从2开始
        for i, (img, label) in enumerate(zip(train_set[0], train_set[1])):
            img_path = data_path+str(i)+'.jpg'
            # 保存图片
            io.imsave(img_path, img.numpy())
            # 保存标号文件路径和标号
            f.write(img_path + ' ' + str(label.item()) + '\n')

        f.close()
    else:
        f = open('test.txt', 'w')
        data_path ='test/'
        if not os.path.exists(data_path):
            os.makedirs(data_path)
        for i, (img, label) in enumerate(zip(test_set[0], test_set[1])):
            img_path = data_path + str(i) + '.jpg'
            io.imsave(img_path, img.numpy())
            f.write(img_path + ' ' + str(label.item()) + '\n')
        f.close()


if os.path.exists('train'): #如果目录不存在就返回False
    print("Pictures(train) already converted.")

else:
    
    print("Building training set...")
    convert_to_img(True)
    print("叮！图像转换完成！赞美太阳！")

if os.path.exists('test'):  # 如果目录不存在就返回False
    print("Pictures(test) already converted.")
else:
    print("Building test set...")
    convert_to_img(False)
    print("叮！图像转换完成！赞美太阳！")


Building training set...
叮！图像转换完成！赞美太阳！
Building test set...
叮！图像转换完成！赞美太阳！


In [9]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
def default_loader(path):
    return Image.open(path).convert('RGB')


class MyDataset(Dataset):
    # txt是路径和文件名
    def __init__(self, txt, transform=transforms.ToTensor(), target_transform=None, loader=default_loader):
        fh = open(txt, 'r')  # 只读打开
        imgs = []
        for line in fh:

            line = line.strip('\n')  # 删除 回车
            line = line.rstrip()  # 删除 右侧 空格
            words = line.split()  # 分割：就两列，0列是路径 1列是标号

            imgs.append((words[0], int(words[1])))
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader  # 是个函数

    # train_loader里面的
    def __getitem__(self, index):
        fn, label = self.imgs[index]   # fn是完整路径 label是标号
        img = self.loader(fn)  # 调用上面的default_loader(path) 按照路径读取图片
        if self.transform is not None:
            img = self.transform(img)  # 将图片转换成FloatTensor类型
        return img, label

    def __len__(self):
        return len(self.imgs)

In [10]:
print("Reading train_data...")
train_data = MyDataset(txt='train.txt', transform=transforms.ToTensor())
# from torch.utils.data import Dataset, DataLoader 下面的函数在这里
train_loader = DataLoader(dataset=train_data, batch_size=50, shuffle=True)
print("Reading test_data...")
test_data = MyDataset(txt='test.txt', transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_data, batch_size=50)

Reading train_data...


ValueError: invalid literal for int() with base 10: 'tensor(4)'

In [ ]:
class Net(torch.nn.module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(  # (1,28,28)
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5,
                      stride=1, padding=2),  # (16,28,28)
            # 想要con2d卷积出来的图片尺寸没有变化, padding=(kernel_size-1)/2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)  # (16,14,14)
        )
        self.conv2 = torch.nn.Sequential(
           torch.nn.Conv2d(32, 64, 3, 1, 1),
           torch.nn.ReLU(),
           torch.nn.MaxPool2d(2)
       )
        self.conv3 = torch.nn.Sequential(
           torch.nn.Conv2d(64, 64, 3, 1, 1),
           torch.nn.ReLU(),
           torch.nn.MaxPool2d(2)
       )
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)  # 将（batch，32,7,7）展平为（batch，32*7*7）
        output = self.out(x)
        return output

In [ ]:
#损失
model = Net()

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)

In [ ]:
for epoch in range(EPOCH):
    start = time.time()
    for step, (x, y) in enumerate(train_loader):
        b_x = Variable(x, requires_grad=False)
        b_y = Variable(y, requires_grad=False)
        # if if_use_gpu:
        b_x = b_x
        b_y = b_y
        print(step)
        output = model(b_x)
        loss = loss_function(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 10 == 0:
            print('Epoch:', epoch, '|Step:', step,
                  '|train loss:%d' % loss.item())

#测试
test_output = model(test_x)
pred_y = torch.max(test_output, 1)[1].data.squeeze()
print(pred_y,test_y)
accuracy = sum(pred_y == test_y) / test_y.size(0)
print('Test Acc: %.4f'%accuracy)